# **Homework 1: COVID-19 Cases Prediction (Regression)**

### [作业辅导](https://www.youtube.com/watch?v=iMzxjBDMvac)

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
# check gpu type
!nvidia-smi

Fri Nov 10 22:58:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.13                 Driver Version: 537.13       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   38C    P8               2W /  93W |     94MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Import packages

In [2]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# Feature selecting
import sklearn
from sklearn.feature_selection import SelectKBest,f_regression

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

In [3]:
# playground
# dir方法会返回类中所有的公共方法
# dir(torch)
# dir(torch.cuda.is_available)
# help(torch.cuda.is_available)

# Download data
If the Google Drive links below do not work, you can use the dropbox link below or download data from [Kaggle](https://www.kaggle.com/competitions/ml2023spring-hw1/overview), and upload data manually to the workspace.

In [4]:
# google drive link
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
# wget方法会报错，经查资料这个wget是linux系统上的方法，由于现在电脑是windows无法调用，后续在mac电脑上尝试
# 在colab上可以执行
# !wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
# !wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0
# /kaggle/input/hw1-covid-19

File_Path_Train = './covid_train.csv'
File_Path_Test = './covid_test.csv'

# Kaggle
# File_Path_Train = '/kaggle/input/hw1-covid-19/covid_train.csv'
# File_Path_Test = '/kaggle/input/hw1-covid-19/covid_test.csv'

# Common
ds_train = pd.read_csv(File_Path_Train)
ds_test = pd.read_csv(File_Path_Train)

### Data checking
先检查一遍数据，按照作业要求，基础数据中，应该包含了美国35个州过去3天的感染人数数据：

- 先尝试看懂数据
- 如果有数据需要padding，先做padding

In [5]:
# ds_train.head()

In [6]:
# ds_train.describe()

In [7]:
# 查看所有列
# for idx,name in enumerate(ds_train.columns):
#   print(idx,name)

In [8]:
# ds_train.shape[0]

In [9]:
# print('--------------------------- STATE ----------------------------------')
# print('id:',ds_train.columns.get_loc('id'))
# print('AL:',ds_train.columns.get_loc('AL'))
# print('WI:',ds_train.columns.get_loc('WI'))
# print('--------------------------- DAY 1 ----------------------------------')
# print('cli:',ds_train.columns.get_loc('cli'))
# print('tested_positive:',ds_train.columns.get_loc('tested_positive'))
# print('--------------------------- DAY 2 ----------------------------------')
# print('cli.1:',ds_train.columns.get_loc('cli.1'))
# print('tested_positive.1:',ds_train.columns.get_loc('tested_positive.1'))
# print('--------------------------- DAY 3 ----------------------------------')
# print('cli.2:',ds_train.columns.get_loc('cli.2'))
# print('tested_positive.2:',ds_train.columns.get_loc('tested_positive.2'))

#### 检查是否包含35个州?
不是35个州，一共34个州。从col[1]到col[34]，AL->WI，
> PS: 美国一共50个州 https://zh.wikipedia.org/zh-hans/%E7%BE%8E%E5%9B%BD%E5%B7%9E%E4%BB%BD

#### 检查是否包含3天的数据?
有三天的数据，每一天有18列数据
- 第一天的数据从col[35]到col[52],从cli到tested_positive
- 第二天有.1的尾缀，从col[53]到col[70],从cli.1到tested_positive.1
- 第三天有.2的尾缀，从col[71]到col[88],从cli.2到tested_positive.2

#### 列名分类
|no|col name|remark|
|-----------|-----------|-----------|
|1|COVID-like illness|冠状病毒样疾病|
|2|Behavior indicators|行为指标|
|3|Belief indicators|信念指标|
|4|Mental indicator|心理指标|
|5|Enviromental indicator|环境指标|
|6|Tested Positive Cases|检测呈阳性的病例|

- ![col-feature-desc](./hw-col-feature-desc.jpg)
- ![col-feature-desc-2](./hw-col-feature-desc-2.jpg)
- ![col-feature-desc-3](./hw-col-feature-desc-3.jpg)

#### 列名解释
由于所有列都为缩写，看起来很费劲，这里找到了一份调查，其中有对应翻译其内容：
https://cmu-delphi.github.io/delphi-epidata/api/covidcast-signals/fb-survey.html

第一个字母为*w*的，代表百分比，否则代表数量

简要摘录内容如下:

|no|col name|desc|Chinese desc|Classification|
|-----------|-----------|-----------|-----------|-----------|
|1|cli|COVID-like illness|COVID-like illness新冠疾病|COVID-like illness|
|2|ili|influenza-like illness|流感样疾病|COVID-like illness|
|3|wnohh_cmnty_cli|not including household community COVID-like illness|社区中非家庭中统计到的cli比例|COVID-like illness|
|4|wbelief_masking_effective||相信戴口罩是有效的预防手段|Belief indicators|
|5|wbelief_distancing_effective||相信远距离是有效的预防手段|Belief indicators|
|6|wcovid_vaccinated_friends||他们的大多数朋友和家人都接种了新冠肺炎疫苗|Behavior indicators|
|7|wlarge_event_indoors||参与过大于10人活动的百分比|Behavior indicators|
|8|wothers_masked_public||受访者表示除他之外 ，大多数人或所有人在公共场合都戴口罩的估计百分比|Enviromental indicator|
|9|wothers_distanced_public||受访者表示除他之外 ，大多数人或所有人在公共场所与他们保持至少为6英尺（1.8米）的估计百分比|Enviromental indicator|
|10|wshop_indoors||去过室内市场、杂货店或药店的受访者的估计百分比|Behavior indicators|
|11|wrestaurant_indoors||去过室内“酒吧、餐厅或咖啡馆”的受访者的估计百分比|Behavior indicators|
|12|wworried_catch_covid||对新冠肺炎感到非常或适度担忧的受访者的估计百分比|Mental indicator|
|13|hh_cmnty_cli||在当地社区报告新冠的人数包括他们的家人|COVID-like illness|
|14|nohh_cmnty_cli||在当地社区报告新冠的人数，不包括他们的家人|COVID-like illness|
|15|wearing_mask_7d||过去7天内大部分或所有时间在公共场合戴口罩的人|Enviromental indicator|
|16|public_transit||使用公共交通|Behavior indicators|
|17|worried_finances||表示非常担心或有点担心下个月家庭财务状况的受访者|Mental indicator|
|18|tested_positive||检测呈阳性的病例|Tested Positive Cases|

In [10]:
# print('--------------------------- COVID-like illness ----------------------------------')
# print(ds_train.loc[:0,['id','cli','ili','wnohh_cmnty_cli','hh_cmnty_cli','nohh_cmnty_cli']])
# print('--------------------------- Behavior indicators ----------------------------------')
# print(ds_train.loc[:0,['id','wcovid_vaccinated_friends','wlarge_event_indoors','wshop_indoors','wrestaurant_indoors','public_transit']])
# print('--------------------------- Belief indicators ----------------------------------')
# print(ds_train.loc[:0,['id','wbelief_masking_effective','wbelief_distancing_effective']])
# print('--------------------------- Mental indicator ----------------------------------')
# print(ds_train.loc[:0,['id','wworried_catch_covid','worried_finances']])
# print('--------------------------- Enviromental indicator ----------------------------------')
# print(ds_train.loc[:0,['id','wothers_masked_public','wothers_distanced_public','wearing_mask_7d']])
# print('--------------------------- Tested Positive Cases ----------------------------------')
# print(ds_train.loc[:0,['id','tested_positive',]])

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 'select_features': np.r_[1:34,35:37,40:45,47:50,52,53:55,58:63,65:68,70,71:73,76:81,83:86], # Select columns with out id, Mental indicator
config = {
    'model_use_business': False,# 设计模型的时候，神经元数量的确实方式是按照feature数量来，还是按照2的N次方来
    'seed': 19871201,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'select_features': [35,36,37,47,48,52,53,54,55,65,66,70,71,72,73,83,84],
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-3,
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.
    'weight_decay': 1e-6,
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

## 记录一些实验结果

### Epoch [1000/1000]: Train loss: 0.8504, Valid loss: 0.8970
将model的layer改为2的n次方，感觉效果一般
```python
# 将神经元得与业务解耦，将业务体现在input_dim,也就是feature_select上。
# 下方linear的神经元数量修改为2的n次方
layer_1_nodes = 64
layer_2_nodes = 32
layer_3_nodes = 16

class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, layer_1_nodes),
            nn.ReLU(),
            nn.Linear(layer_1_nodes, layer_2_nodes),
            nn.ReLU(),
            nn.Linear(layer_2_nodes, layer_3_nodes),
            nn.ReLU(),
            nn.Linear(layer_3_nodes, 1) # 最后归为一个阳性的数量
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x
```

### Epoch [1000/1000]: Train loss: 0.8320, Valid loss: 0.8807

选feature时，更新了feature的筛选，将beilf和mental都去掉。同时，将feature放入config，这需要修改train时的代码
```python
config = {
    'seed': 19871201,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'select_features': np.r_[1:34,35:37,40:45,47:50,52,53:55,58:63,65:68,70,71:73,76:81,83:86], # Select columns with out id, Mental indicator
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 1000,     # Number of epochs.
    'batch_size': 34*10,
    'learning_rate': 1e-3,
    'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.
    'weight_decay': 1e-6,
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'], config['select_features'])
```

optimizer使用了Adam方法，并进行L2 Regularzation
```python
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], betas=(0.9, 0.999), eps=1e-08, weight_decay=config['weight_decay'], amsgrad=False)
```

### Epoch [1000/1000]: Train loss: 1.1560, Valid loss: 1.3151

```python
# define feature
state_count = 34
feature_count = 15
feature_type_count = 4 # 去掉了一个心理因素
neuron_nodes = state_count + feature_count*3 # 34个州，一共有15列有用特征*3天

nn.Sequential(
    nn.Linear(input_dim, neuron_nodes),
    nn.ReLU(),
    nn.Linear(neuron_nodes, feature_count),
    nn.ReLU(),
    nn.Linear(feature_count, feature_type_count),
    nn.ReLU(),
    nn.Linear(feature_type_count, 1) # 最后归为一个阳性的数量
)

feat_idx = np.r_[1:34,35:45,47:50,52,53:63,65:68,70,71:81,83:86] # Select columns with out id, Mental indicator
```

### Epoch [1000/1000]: Train loss: 4.8335, Valid loss: 3.4793

```python
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 17),
            nn.ReLU(),
            nn.ReLU(),
            nn.Linear(17, 5),
            nn.ReLU(),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

```

### 下面这个原生网络的结果
修改batch之后：


Epoch [1000/1000]: Train loss: 5.5759, Valid loss: 2.4200

Epoch [4569/5000]: Train loss: 1.2975, Valid loss: 1.9382

``` python
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x
```

# Some Utility Functions

You do not need to modify this part.

In [12]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.切换模型状态，后面还有model.train()，要来回切换使用，挺麻烦的
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

In [13]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [14]:
# @title
# 定义nn(nerual network) 一般需要重写init和forward方法
# forward是前馈网络的意思

# 将神经元得与业务解耦，将业务体现在input_dim,也就是feature_select上
# 下方linear的神经元数量修改为2的n次方
layer_1_nodes = 16
layer_2_nodes = 8
layer_3_nodes = 4

if config['model_use_business']:
  # define feature
  state_count = 34
  previous_count = 2
  feature_count = 13
  feature_type_count = 3 # 去掉了心理因素，相信因素
  neuron_nodes = feature_count*3 # 34个州，一共有15列有用特征*3天

  layer_1_nodes = neuron_nodes + previous_count
  layer_2_nodes = feature_count+1 + previous_count
  layer_3_nodes = feature_type_count+ previous_count

class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, layer_1_nodes),
            nn.ReLU(),
            nn.Linear(layer_1_nodes, layer_2_nodes),
            nn.ReLU(),
            nn.Linear(layer_2_nodes, layer_3_nodes),
            nn.ReLU(),
            nn.Linear(layer_3_nodes, 1) # 最后归为一个阳性的数量
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [15]:
# np.r_[35:37,40:45,47:50,52,53:55,58:63,65:68,70,71:73,76:81,83:86], # 往期自定义的结果


train_data_for_select = pd.read_csv(File_Path_Train).values
# train_data_for_select, valid_data = train_valid_split(train_data_for_select, config['valid_ratio'], config['seed'])
train_data_for_select_X,train_data_for_select_Y = train_data_for_select[:,:-1],train_data_for_select[:,-1]


# k设置为17，其预估是将前两天的阳性人数和3天中相关度比较的因素取前5，最终返回的列如下：
# cli   ili   wnohh_cmnty_cli   hh_cmnty_cli   nohh_cmnty_cli   tested_positive
# cli.1 ili.1 wnohh_cmnty_cli.1 hh_cmnty_cli.1 nohh_cmnty_cli.1 tested_positive.1
# cli.2 ili.2 wnohh_cmnty_cli.2 hh_cmnty_cli.2 nohh_cmnty_cli.2

# k设置为2，只取两个因素，其方法返回的值为：
# tested_positive，tested_positive.1

# 从结果上看，返回的内容很不错

def select_feat_sklearn(x,y):
 k=17
 x_new = SelectKBest(score_func=f_regression,k=k).fit(x,y)
 return x_new

# print(train_data_for_select_X[:1,:])
# train_data_for_select_X.shape
# train_data_for_select_Y.shape
# print(train_data_for_select_Y)

features = select_feat_sklearn(train_data_for_select_X,train_data_for_select_Y)
array = features.get_feature_names_out()
for idx,name in enumerate(array):
 array[idx] = int(array[idx][1:])

# [35 36 37 47 48 52 53 54 55 65 66 70 71 72 73 83 84]
    
print(array)
# idx = np.argsort(features.scores_)[::1]
# print(features[:3])


[35 36 37 47 48 52 53 54 55 65 66 70 71 72 73 83 84]


In [16]:
# 参数select_all默认为True，代表默认会将所有列作为feature加入计算
# 如果在训练过程中，有发现某几列可能关联性不大，可以排除出来，那要将select_all改为false，并手工导入列数
def select_feat(train_data, valid_data, test_data, select_all=True, select_features=[]):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = select_features

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# Training Loop

In [17]:
same_seed(config['seed'])

train_data, test_data = pd.read_csv(File_Path_Train).values, pd.read_csv(File_Path_Test).values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'], config['select_features'])

print('x_train')
print(x_train)

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2408, 89)
valid_data size: (601, 89)
test_data size: (997, 88)
x_train
[[ 1.3138593  1.2748362 15.7057702 ... 15.8342843 22.0118552 16.1409036]
 [ 1.6800285  1.7248551 16.6408925 ... 16.2795037 21.1383018 16.1498569]
 [ 0.8818757  0.9345295  9.7551765 ...  8.7525655 12.8101195  8.8501928]
 ...
 [ 1.0946756  1.1118771 11.7432324 ... 12.0424304 15.3299617 10.9991829]
 [ 3.6352241  3.7769054 26.5714705 ... 23.7412583 32.9247972 25.8964017]
 [ 3.6850859  3.7616652 33.6158757 ... 31.6886024 37.331582  31.3212202]]
number of features: 17


In [21]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], betas=(0.9, 0.999), eps=1e-08, weight_decay=config['weight_decay'], amsgrad=False)
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        # writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        if(epoch%100 == 0):
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'] + str(best_loss)) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            print('Model current loss {:.3f}'.format(best_loss))
            return

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

# Start training!

In [22]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: 100%|██████████| 10/10 [00:00<00:00, 187.58it/s, loss=357]


Epoch [1/5000]: Train loss: 358.7049, Valid loss: 336.1781
Saving model with loss 336.178...


Epoch [2/5000]: 100%|██████████| 10/10 [00:00<00:00, 239.50it/s, loss=246]


Saving model with loss 275.893...


Epoch [3/5000]: 100%|██████████| 10/10 [00:00<00:00, 192.65it/s, loss=195]


...

Epoch [1401/5000]: Train loss: 0.8830, Valid loss: 0.8552


Epoch [1501/5000]: 100%|██████████| 10/10 [00:00<00:00, 252.72it/s, loss=0.785]


Epoch [1501/5000]: Train loss: 0.8867, Valid loss: 0.8703


Epoch [1592/5000]: 100%|██████████| 10/10 [00:00<00:00, 263.91it/s, loss=1.21]


Model is not improving, so we halt the training session.
Model current loss 0.775


### 验证模型结果
模型是loss在0.77左右存档的，用该模型重新对验证集(valid)进行验证，观察loss是否仍然是0.77

In [24]:
same_seed(config['seed'])
train_data, test_data = pd.read_csv(File_Path_Train).values, pd.read_csv(File_Path_Test).values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# # Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'], config['select_features'])

valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

criterion = nn.MSELoss(reduction='mean')

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
loss_record = []
for x, y in valid_loader:
    x, y = x.to(device), y.to(device)
    with torch.no_grad():
        pred = model(x)
        loss = criterion(pred, y)

    loss_record.append(loss.item())

mean_valid_loss = sum(loss_record)/len(loss_record)
print(mean_valid_loss)

0.8618664741516113


# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

# Download

Run this block to download the `pred.csv` automatically.

In [ ]:
# from google.colab import files
# files.download('pred.csv')

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)